In [ ]:
# Import packages
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
import numpy as np

In [3]:
# Load Data
data_filename =  '/PATH/TO/all_subs_data_w_footstrike.pkl'

df = np.load(data_filename, allow_pickle=True)
feats = df['train_feats']  # (trials, frames per trial, features)
y = df['train_y']  # (trials, frames per trial)
sub_info = df['train_Sub_Info']  # (trials, features)

# Test LSTM on Representative Subject (#14 when sorted from lowest -> highest RMSE in paper). 
# Expect Validation MSE ~ 0.03 BW (RMSE = 0.17 BW).
sub_num = 2

test_X = feats[sub_info['Sub'] == sub_num,:,:]
test_y = y[sub_info['Sub'] == sub_num,:]

train_X = feats[sub_info['Sub'] != sub_num,:,:] 
train_y = y[sub_info['Sub'] != sub_num,:]

In [4]:
# Make sure GPU runtime is activated before training LSTM!

# Build Model
def build_model(lstm_size, lstm_act, dropout_rate, dense_act, lr=0.001, loss='mean_squared_error'):

  #accelerometer data lstm model
  model_inputs = keras.Input(shape=(None,train_X.shape[2]))
  model_features = keras.layers.Dropout(0.2, seed=541)(model_inputs)
  model_features = keras.layers.Bidirectional(keras.layers.LSTM(lstm_size, activation=lstm_act, return_sequences=True), merge_mode='ave')(model_features)
  model_features = keras.layers.Dropout(dropout_rate, seed=541)(model_features)
  model_features = keras.layers.Dense(128, activation=dense_act)(model_features)
  model_features = keras.layers.Dense(384, activation=dense_act)(model_features)
  model_features = keras.layers.Dense(320, activation=dense_act)(model_features)
  model_outputs = keras.layers.Dense(1, activation='linear')(model_features)

  model_out = keras.Model(inputs=model_inputs, outputs=model_outputs, name='LSTM')
  # define optimizer algorithm and learning rate
  opt = keras.optimizers.Adam(learning_rate =lr)
  # compile model and define loss function
  model_out.compile(optimizer=opt, loss=loss)

  return model_out

model = build_model(
    lstm_size=512,
    lstm_act='tanh',
    dropout_rate=0.4,
    dense_act='relu',
    lr=0.001
    )

# Plot Model Architecture
# keras.utils.plot_model(model, show_shapes=True, show_layer_names=True)

In [ ]:
# Train Model

# Define Early Stopping and Checkpoint Callbacks
model_filename = '/PATH/TO/MODEL.h5'

# Early Stopping
es = keras.callbacks.EarlyStopping(monitor='val_loss', 
                                  mode='min', 
                                  verbose=0, 
                                  patience=30, 
                                  min_delta=0.001, 
                                  restore_best_weights=True
                                  )
# Model Checkpoint
mc = keras.callbacks.ModelCheckpoint(
    model_filename,
    monitor='val_loss', 
    mode='min', 
    verbose=1, 
    save_best_only=True, 
    save_weights_only=False
    )

# Fit Model
history_accel = model.fit(
    train_X, 
    train_y, 
    epochs=1000,
    validation_data=(test_X, test_y), 
    verbose=1,
    batch_size=32, 
    callbacks=[es, mc]
    )

# Plot Train/Validation Loss across epochs
plt.plot(history_accel.history['loss'], label = 'mse_train')
plt.plot(history_accel.history['val_loss'], label = 'mse_validation')
plt.legend()
plt.show()

keras.backend.clear_session()

In [9]:
# Load Trained Model and Calculate RMSE for GRF Waveform
saved_model = keras.models.load_model(model_filename)

pred_y = saved_model.predict(test_X)
test_sub_info = sub_info.loc[sub_info['Sub'] == sub_num,:]

test_y = np.squeeze(test_y)
pred_y = np.squeeze(pred_y)

rmse = []
trim = 100  # Number of frames to ignore at edge of trial due to lack of prior data for LSTM.

for trial in range(test_sub_info.shape[0]):
  # Calculate RMSE
  trial_rmse = np.sqrt(np.mean((pred_y[trial, trim:-trim] - test_y[trial, trim:-trim])**2))
  trial_rmse = np.round(trial_rmse,3)
  # Calculate rRMSE
  trial_rrmse = trial_rmse / np.mean((
      np.max(pred_y[trial, trim:-trim]) - np.min(pred_y[trial, trim:-trim]),
      np.max(test_y[trial, trim:-trim]) - np.min(test_y[trial, trim:-trim])
      ))*100
  trial_rrmse = np.round(trial_rrmse, 2)

  rmse.append(trial_rmse)

# Expect RMSE of approximately 0.17 ± 0.07 BW for Representative Subject (#14 in paper)
print('MEAN:', np.round(np.mean(rmse),2))
print('SD:', np.round(np.std(rmse),2))

MEAN: 0.16
SD: 0.05
